In [1]:
import sys
import os
BASE_PATH = os.path.abspath('../..')
sys.path.append(BASE_PATH)

import logging
from work.auxiliary.logger_settings import configure_logger
from keras.callbacks import ReduceLROnPlateau
from work.auxiliary import data_functions

from work.unet import unet_model_functions 

Using TensorFlow backend.


In [2]:
LOG_PATH = os.path.join(BASE_PATH,'logs')
DATA_PATH = os.path.join(BASE_PATH,'data') # create a data folder at the save root folder with the "work" folder

log_path = data_functions.create_path(LOG_PATH, 'unet_logs')

configure_logger(name="cherry_stem",
                 console_level='INFO',
                 file_level='INFO',
                 out_path=log_path)

logger = logging.getLogger(__name__)

## Define paths

In [3]:
train_path = os.path.join(DATA_PATH,r'segmentation\augmented') #path where the "label" and "image" folder resides
dest_path =  os.path.join(DATA_PATH,r'unet_data\training') # the path to save the current session results
test_path = os.path.join(DATA_PATH,r'raw_data\images_orig') # path containig images to segment

pretrained_weights = os.path.join(DATA_PATH, r'unet_data\training\2019-09-30_07-19-46\unet_cherry_stem.hdf5')
checkpoint=

src_path = os.path.join(BASE_PATH,r'data\unet_data\training\2019-10-15_16-21-49') # optional path to load a pretrained model
                                                                                  # to continue trainig
steps=None # number of steps to select where to continue the trainig from

## Model parameters example

In [4]:
params_dict = dict(

    train_path=train_path,
    save_path=dest_path,
    x_folder_name='image', # the name of the 'image' folder within the train path (contains the X data or images)
    y_folder_name='label', # the name of the 'label' folder within the train path (contains the Y data or labels)

    save_name='cherry', # name used as basename for all output which is save (the base file name)

    pretrained_weights=pretrained_weights, # path to pretrained wights which can be loaded for this training session
    checkpoint=None, #optional, path to pretrained folder of pretrained model which can be loaded for this training session
    data_gen_args=dict(rotation_range=180,
                       brightness_range=[0.2, 1.],
                       shear_range=5,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'),# augmentaion arguments for the imageDataGen module in keras
    seed=78,# Random seed used in the generation of the train images

    optimizer='Adam',# name of the optimizer to use, can be any of the optimizer in: https://keras.io/optimizers/
    optimizer_params=dict(lr=1e-5),

    loss='binary_crossentropy',
    metrics=[],# list of metrics which will be monitored during trainig

    target_size=(256, 256), # the source images are resized for computetional efficiency to this target size
    color_mode='grayscale', # where to train of rgb images or convert to grayscale

    batch_size=10,  # my GPU cant handel any more

    epochs=10,
    steps_per_epoch=3000,
    validation_split=0.2,
    validation_steps=300,

    tensorboard_update_freq=100, #update frequency for the metrics and iamges displayed in tensorboard
    weights_update_freq='epoch', #frequency to save the model to chekpoint, if 'epoch' than after each trainig epoch
    save_weights_only=True, # whether to save just the model weights or the model itself
    ontop_display_threshold=0.4, # creates binary masks for model predicionts with pixel values greater than this, which are
    #shown in tensorboard
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                                 verbose=1, mode='auto', min_delta=0.0001,
                                 cooldown=0, min_lr=1e-6)])


In [ ]:
#load exsisting model

update_dict = dict(tensorboard_update_freq=5000,
                   weights_update_freq=20000,
                   optimizer_params=dict(lr=1e-6)) # optional dictionay to update the loaded model parameters 

model = unet_model_functions.ClarifruitUnet.load_model(src_path,steps)
model.update_model(**update_dict)

In [5]:

model = unet_model_functions.ClarifruitUnet(**params_dict)

2019-10-22 18:22:01,005-6d  INFO     work.unet.unet_model_functions: loading model weights from D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-09-30_07-19-46\unet_cherry_stem.hdf5
2019-10-22 18:22:01,021-6d  INFO     work.unet.unet_model: creating unet model with input_size=(256, 256, 1)
2019-10-22 18:22:01,301-6d  INFO     work.unet.unet_model: loading unet weights: D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-09-30_07-19-46\unet_cherry_stem.hdf5


In [6]:
keras_logs_path=model.set_model_for_train()
print(f"tensorboard --logdir={keras_logs_path}") #commad for activating tensorboard for this session

2019-10-22 18:22:04,346-6d  INFO     work.unet.unet_model_functions: training results will be stored in: D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
tensorboard --logdir=D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04\keras_logs


In [7]:
model.fit_unet()

2019-10-22 18:22:13,180-6d  INFO     work.unet.unet_model_functions: Training model with optimizer:Adam
2019-10-22 18:22:13,180-6d  INFO     work.unet.unet_model_functions: Optimizer params: {'lr': 1e-05}
Epoch 1/10
3000/3000 [==============================] - 3039s 1s/step - loss: 0.0336 - val_loss: 0.0584

Epoch 00001: saving model to D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04\cherry_weights.train_sess_01.steps_28504.loss_0.0336.hdf5
Epoch 2/10
3000/3000 [==============================] - 3078s 1s/step - loss: 0.0312 - val_loss: 0.0554

Epoch 00002: saving model to D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04\cherry_weights.train_sess_01.steps_57000.loss_0.0312.hdf5
Epoch 3/10
3000/3000 [==============================] - 3166s 1s/step - loss: 0.0304 - val_loss: 0.0548

Epoch 00003: saving model to D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04\cherry_weights.train_s

In [9]:
model.model.fit

2019-10-23 06:52:53,653-6d  INFO     work.unet.unet_model_functions: Training model with optimizer:Adam
2019-10-23 06:52:53,653-6d  INFO     work.unet.unet_model_functions: Optimizer params: {'lr': 1e-05}


ValueError: not enough values to unpack (expected 3, got 1)

In [8]:
model.prediction(test_path,dest_path)


2019-10-23 06:46:43,534-6d  INFO     work.unet.unet_model_functions: prediction on files from D:\Files\Projects\Clarifruit\Cherry_stem\data\raw_data\images_orig
2019-10-23 06:46:43,688-6d  INFO     work.unet.unet_model_functions: saving predictions to D:\Files\Projects\Clarifruit\Cherry_stem\data\unet_data\training\2019-10-22_18-22-04\raw_pred
2019-10-23 06:46:44,337-6d  INFO     work.unet.unet_model_functions: getting prediction for 38357-02789.png.jpg
2019-10-23 06:46:46,499-6d  INFO     work.unet.unet_model_functions: getting prediction for 38357-12440.png.jpg
2019-10-23 06:46:46,671-6d  INFO     work.unet.unet_model_functions: getting prediction for 38357-13707.png.jpg
2019-10-23 06:46:46,801-6d  INFO     work.unet.unet_model_functions: getting prediction for 38357-27009.png.jpg
2019-10-23 06:46:46,857-6d  INFO     work.unet.unet_model_functions: getting prediction for 38357-42160.png.jpg
2019-10-23 06:46:46,942-6d  INFO     work.unet.unet_model_functions: getting prediction for 38

2019-10-23 06:46:51,112-6d  INFO     work.unet.unet_model_functions: getting prediction for 72515-98438.png.jpg
2019-10-23 06:46:51,165-6d  INFO     work.unet.unet_model_functions: getting prediction for 72520-70104.png.jpg
2019-10-23 06:46:51,216-6d  INFO     work.unet.unet_model_functions: getting prediction for 72590-54586.png.jpg
2019-10-23 06:46:51,268-6d  INFO     work.unet.unet_model_functions: getting prediction for 72592-11978.png.jpg
2019-10-23 06:46:51,322-6d  INFO     work.unet.unet_model_functions: getting prediction for 72596-28736.png.jpg
2019-10-23 06:46:51,376-6d  INFO     work.unet.unet_model_functions: getting prediction for 74714-32897.png.jpg
2019-10-23 06:46:51,430-6d  INFO     work.unet.unet_model_functions: getting prediction for 74716-02522.png.jpg
2019-10-23 06:46:51,510-6d  INFO     work.unet.unet_model_functions: getting prediction for 74717-45732.png.jpg
2019-10-23 06:46:51,563-6d  INFO     work.unet.unet_model_functions: getting prediction for 74719-86289.

2019-10-23 06:46:56,201-6d  INFO     work.unet.unet_model_functions: getting prediction for 75291-43129.png.jpg
2019-10-23 06:46:56,253-6d  INFO     work.unet.unet_model_functions: getting prediction for 75292-17661.png.jpg
2019-10-23 06:46:56,322-6d  INFO     work.unet.unet_model_functions: getting prediction for 75294-22028.png.jpg
2019-10-23 06:46:56,374-6d  INFO     work.unet.unet_model_functions: getting prediction for 75296-34318.png.jpg
2019-10-23 06:46:56,442-6d  INFO     work.unet.unet_model_functions: getting prediction for 75297-20819.png.jpg
2019-10-23 06:46:56,493-6d  INFO     work.unet.unet_model_functions: getting prediction for 75301-00044.png.jpg
2019-10-23 06:46:56,546-6d  INFO     work.unet.unet_model_functions: getting prediction for 75304-60632.png.jpg
2019-10-23 06:46:56,597-6d  INFO     work.unet.unet_model_functions: getting prediction for 75306-20296.png.jpg
2019-10-23 06:46:56,650-6d  INFO     work.unet.unet_model_functions: getting prediction for 75312-89260.

2019-10-23 06:47:01,694-6d  INFO     work.unet.unet_model_functions: getting prediction for 75425-51236.png.jpg
2019-10-23 06:47:01,850-6d  INFO     work.unet.unet_model_functions: getting prediction for 75427-40868.png.jpg
2019-10-23 06:47:01,937-6d  INFO     work.unet.unet_model_functions: getting prediction for 75428-84476.png.jpg
2019-10-23 06:47:01,989-6d  INFO     work.unet.unet_model_functions: getting prediction for 75430-57591.png.jpg
2019-10-23 06:47:02,041-6d  INFO     work.unet.unet_model_functions: getting prediction for 75431-38492.png.jpg
2019-10-23 06:47:02,093-6d  INFO     work.unet.unet_model_functions: getting prediction for 75917-07816.png.jpg
2019-10-23 06:47:02,144-6d  INFO     work.unet.unet_model_functions: getting prediction for 75922-86763.png.jpg
2019-10-23 06:47:02,196-6d  INFO     work.unet.unet_model_functions: getting prediction for 75926-39221.png.jpg
2019-10-23 06:47:02,249-6d  INFO     work.unet.unet_model_functions: getting prediction for 75929-67046.

2019-10-23 06:47:07,906-6d  INFO     work.unet.unet_model_functions: getting prediction for 77830-24394.png.jpg
2019-10-23 06:47:08,001-6d  INFO     work.unet.unet_model_functions: getting prediction for 77830-36229.png.jpg
2019-10-23 06:47:08,095-6d  INFO     work.unet.unet_model_functions: getting prediction for 77830-41305.png.jpg
2019-10-23 06:47:08,375-6d  INFO     work.unet.unet_model_functions: getting prediction for 77830-51006.png.jpg
2019-10-23 06:47:08,563-6d  INFO     work.unet.unet_model_functions: getting prediction for 78233-08344.png.jpg
2019-10-23 06:47:08,717-6d  INFO     work.unet.unet_model_functions: getting prediction for 78569-84989.png.jpg
2019-10-23 06:47:08,838-6d  INFO     work.unet.unet_model_functions: getting prediction for 78571-78013.png.jpg
2019-10-23 06:47:08,891-6d  INFO     work.unet.unet_model_functions: getting prediction for 78573-28316.png.jpg
2019-10-23 06:47:08,944-6d  INFO     work.unet.unet_model_functions: getting prediction for 78583-84075.

2019-10-23 06:47:14,610-6d  INFO     work.unet.unet_model_functions: getting prediction for 79638-56334.png.jpg
2019-10-23 06:47:14,668-6d  INFO     work.unet.unet_model_functions: getting prediction for 79640-73349.png.jpg
2019-10-23 06:47:14,724-6d  INFO     work.unet.unet_model_functions: getting prediction for 79641-05305.png.jpg
2019-10-23 06:47:14,784-6d  INFO     work.unet.unet_model_functions: getting prediction for 79642-92430.png.jpg
2019-10-23 06:47:14,848-6d  INFO     work.unet.unet_model_functions: getting prediction for 79644-64685.png.jpg
2019-10-23 06:47:15,066-6d  INFO     work.unet.unet_model_functions: getting prediction for 79645-58016.png.jpg
2019-10-23 06:47:15,184-6d  INFO     work.unet.unet_model_functions: getting prediction for 79764-07824.png.jpg
2019-10-23 06:47:15,241-6d  INFO     work.unet.unet_model_functions: getting prediction for 79766-34317.png.jpg
2019-10-23 06:47:15,298-6d  INFO     work.unet.unet_model_functions: getting prediction for 79767-68821.

2019-10-23 06:47:20,290-6d  INFO     work.unet.unet_model_functions: getting prediction for 80143-73893.png.jpg
2019-10-23 06:47:20,345-6d  INFO     work.unet.unet_model_functions: getting prediction for 80145-71093.png.jpg
2019-10-23 06:47:20,562-6d  INFO     work.unet.unet_model_functions: getting prediction for 80147-64635.png.jpg
2019-10-23 06:47:20,829-6d  INFO     work.unet.unet_model_functions: getting prediction for 80149-29234.png.jpg
2019-10-23 06:47:20,883-6d  INFO     work.unet.unet_model_functions: getting prediction for 80254-15930.png.jpg
2019-10-23 06:47:20,937-6d  INFO     work.unet.unet_model_functions: getting prediction for 80255-24417.png.jpg
2019-10-23 06:47:20,991-6d  INFO     work.unet.unet_model_functions: getting prediction for 80256-29670.png.jpg
2019-10-23 06:47:21,048-6d  INFO     work.unet.unet_model_functions: getting prediction for 80257-55775.png.jpg
2019-10-23 06:47:21,103-6d  INFO     work.unet.unet_model_functions: getting prediction for 80258-59160.

2019-10-23 06:47:25,596-6d  INFO     work.unet.unet_model_functions: getting prediction for 81024-70882.png.jpg
2019-10-23 06:47:25,650-6d  INFO     work.unet.unet_model_functions: getting prediction for 81025-14574.png.jpg
2019-10-23 06:47:25,703-6d  INFO     work.unet.unet_model_functions: getting prediction for 81027-91960.png.jpg
2019-10-23 06:47:25,756-6d  INFO     work.unet.unet_model_functions: getting prediction for 81028-53774.png.jpg
2019-10-23 06:47:25,808-6d  INFO     work.unet.unet_model_functions: getting prediction for 81029-40416.png.jpg
2019-10-23 06:47:25,972-6d  INFO     work.unet.unet_model_functions: getting prediction for 81030-34753.png.jpg
2019-10-23 06:47:26,065-6d  INFO     work.unet.unet_model_functions: getting prediction for 81031-55652.png.jpg
2019-10-23 06:47:26,117-6d  INFO     work.unet.unet_model_functions: getting prediction for 81033-19881.png.jpg
2019-10-23 06:47:26,171-6d  INFO     work.unet.unet_model_functions: getting prediction for 81034-64764.

2019-10-23 06:47:32,012-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-13660.png.jpg
2019-10-23 06:47:32,065-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-15055.png.jpg
2019-10-23 06:47:32,118-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-16070.png.jpg
2019-10-23 06:47:32,170-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-27557.png.jpg
2019-10-23 06:47:32,222-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-31728.png.jpg
2019-10-23 06:47:32,273-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-38440.png.jpg
2019-10-23 06:47:32,324-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-41235.png.jpg
2019-10-23 06:47:32,375-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-48682.png.jpg
2019-10-23 06:47:32,427-6d  INFO     work.unet.unet_model_functions: getting prediction for 81327-51241.

'D:\\Files\\Projects\\Clarifruit\\Cherry_stem\\data\\unet_data\\training\\2019-10-22_18-22-04\\raw_pred'